In [156]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import CactusDataset
from sklearn.metrics import roc_curve, auc


In [71]:
#use_gpu = torch.cuda.is_available()
#if use_gpu:
#    print("Using CUDA")

In [157]:
data_dir = '../../../data/train/train'
csv_file = '../../../data/train.csv'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

#set the transforms for the images
data_transforms = transforms.Compose([
    transforms.ToTensor(),
])


np.random.seed(0)
image_dataset = CactusDataset.CactusDataset(csv_file, data_dir, data_transforms)
# split into train, eval, test
train_size = int(0.7 * len(image_dataset))
eval_size = int(0.2 * len(image_dataset))
test_size = len(image_dataset) - train_size - eval_size
train_dataset, eval_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_size, eval_size, test_size])

image_datasets = {TRAIN: train_dataset, VAL: eval_dataset, TEST: test_dataset}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

batch_sizes = {TRAIN: dataset_sizes[TRAIN], VAL: dataset_sizes[VAL], TEST: 64}

dataloaders = {TRAIN: None, VAL: None, TEST: None}
dataloaders[TRAIN] = DataLoader(image_datasets[TRAIN], batch_size=batch_sizes[TRAIN],
                                             shuffle=True, num_workers=0)
dataloaders[VAL] = DataLoader(image_datasets[VAL], batch_size=batch_sizes[VAL],
                                             shuffle=False, num_workers=0)
dataloaders[TEST] = DataLoader(image_datasets[TEST], batch_size=batch_sizes[TEST], shuffle=False, num_workers=0)


print(dataset_sizes)
class_names = {0: 'No Cactus', 1: 'Cactus'}

inputs, classes = next(iter(dataloaders[TRAIN]))

n_features = 32 * 32 * 3

print(n_features)


{'train': 12250, 'val': 3500, 'test': 1750}
3072


In [158]:
# Create model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

In [159]:
# Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [167]:
def train_model(model, criterion, optimizer, model_name, X_train, y_train, X_val, y_val, num_epochs=1000):
    # training loop
    num_epochs = num_epochs
    accs_train = []
    accs_val = []
    aucs = []
    best_fpr = []
    best_tpr = []
    best_auc = 0
    best_acc = 0
    #40k epoche runnate
    for epoch in range(num_epochs):
        #randomly horizontal flip the images in X_train
        X_train = torch.tensor(X_train)
        transform = transforms.Compose([transforms.RandomHorizontalFlip()])
        X_train = transform(X_train)
        # forward pass and loss
        outputs = model(torch.tensor(X_train))
        #make y_predicted and y_train of same shape
        outputs = outputs.view(-1)

        preds = outputs.round()
        acc_train = (preds == y_train).sum() / y_train.shape[0]
        accs_train.append(acc_train)

        loss = criterion(outputs, y_train.float())
        
        # backward pass
        loss.backward()
        
        # updates
        optimizer.step()
        
        # zero gradients
        optimizer.zero_grad()
        
        with torch.no_grad():
            outputs = model(torch.tensor(X_val))  # no need to call model.forward()
            y_predicted_cls = outputs.round()   # round off to nearest class
            #flatten
            y_val = y_val.view(-1)
            y_predicted_cls = y_predicted_cls.view(-1)
            #Compute accuracy
            acc = (y_predicted_cls == y_val).sum() / y_val.shape[0]   # accuracy
            accs_val.append(acc)
            #Compute AUC
            fpr, tpr, _ = roc_curve(y_val, outputs)
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            if roc_auc > best_auc:
                best_auc = roc_auc
                epoch_best = epoch
                best_fpr = fpr
                best_tpr = tpr
            
    
            if acc > best_acc:
                best_acc = acc
                epoch_best = epoch
                #save the best model weights with info about PCA compoenents
                torch.save(model.state_dict(), f'../../../data/LR/'+str(model_name)+f'weights_best.pt')
            if epoch % 100 == 0:
                print(f'Epoch: {epoch}, Loss: {loss.item()}, Accuracy on val: {acc}')
    return accs_train, accs_val, aucs, best_fpr, best_tpr, best_auc, epoch_best

## LR without PCA

In [161]:
#Load dataset
X_train = []
y_train = []
for images, labels in dataloaders[TRAIN]:
    images = images.view(-1, n_features)
    X_train.append(images)
    y_train.append(labels)
X_train = torch.cat(X_train, dim=0)
y_train = torch.cat(y_train, dim=0)
X_val = []
y_val = []
for images, labels in dataloaders[VAL]:
    images = images.view(-1, n_features)
    X_val.append(images)
    y_val.append(labels)
X_val = torch.cat(X_val, dim=0)
y_val = torch.cat(y_val, dim=0)

In [ ]:
# train model for 1000 epochs
train_model(model, criterion, optimizer, 'logistic_regression', X_train, y_train, X_val, y_val, num_epochs=1000)

## LR with PCA

In [162]:
from sklearn.decomposition import PCA

In [163]:
X_train = []
y_train = []
for images, labels in dataloaders[TRAIN]:
    images = images.view(-1, n_features)
    X_train.append(images)
    y_train.append(labels)
X_train = torch.cat(X_train, dim=0)
y_train = torch.cat(y_train, dim=0)
X_val = []
y_val = []
for images, labels in dataloaders[VAL]:
    images = images.view(-1, n_features)
    X_val.append(images)
    y_val.append(labels)
X_val = torch.cat(X_val, dim=0)
y_val = torch.cat(y_val, dim=0)

In [164]:
# retain only n_componenets feature with highest variance using PCA
n_components = 625
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train.detach().numpy())
X_val_pca = pca.transform(X_val.detach().numpy())

In [165]:
model = LogisticRegression(X_train_pca.shape[1])
# Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [168]:
#Train model using pca features
accs_train, accs_val, aucs, best_fpr, best_tpr, best_auc, epoch_best = train_model(model, criterion, optimizer, 'logistic_regression_pca', X_train_pca, y_train, X_val_pca, y_val, num_epochs=10000)
#plot accuracy train vs val
plt.plot(accs_train, label='train')
plt.plot(accs_val, label='val')
plt.legend()
plt.show()
#plot AUC
plt.plot(aucs)
plt.show()
#plot ROC
plt.plot(best_fpr, best_tpr)
plt.show()
print('Best AUC:', best_auc)
print('Best epoch:', epoch_best)


/tmp/ipykernel_69432/2208340002.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(X_train))
/tmp/ipykernel_69432/2208340002.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)


epoch: 100, loss = 0.5503
tensor(0.7800)


/tmp/ipykernel_69432/2208340002.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
/tmp/ipykernel_69432/2208340002.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(X_train))


epoch: 200, loss = 0.5112
tensor(0.7926)


/tmp/ipykernel_69432/2208340002.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
/tmp/ipykernel_69432/2208340002.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(X_train))


...

epoch: 2200, loss = 0.3261
tensor(0.8726)


/tmp/ipykernel_69432/2208340002.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
/tmp/ipykernel_69432/2208340002.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(X_train))


epoch: 2300, loss = 0.3233
tensor(0.8731)


/tmp/ipykernel_69432/2208340002.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
/tmp/ipykernel_69432/2208340002.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(X_train))


KeyboardInterrupt: 

## LR using only grayscale values

In [113]:
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

data_dir = '../../../data/train/train'
csv_file = '../../../data/train.csv'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'


np.random.seed(0)
image_dataset = CactusDataset.CactusDataset(csv_file, data_dir, data_transforms)
# split into train, eval, test
train_size = int(0.7 * len(image_dataset))
eval_size = int(0.2 * len(image_dataset))
test_size = len(image_dataset) - train_size - eval_size
train_dataset, eval_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_size, eval_size, test_size])

image_datasets = {TRAIN: train_dataset, VAL: eval_dataset, TEST: test_dataset}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

batch_sizes = {TRAIN: dataset_sizes[TRAIN], VAL: dataset_sizes[VAL], TEST: 64}

dataloaders = {TRAIN: None, VAL: None, TEST: None}
dataloaders[TRAIN] = DataLoader(image_datasets[TRAIN], batch_size=batch_sizes[TRAIN],
                                             shuffle=True, num_workers=0)
dataloaders[VAL] = DataLoader(image_datasets[VAL], batch_size=batch_sizes[VAL],
                                             shuffle=False, num_workers=0)
dataloaders[TEST] = DataLoader(image_datasets[TEST], batch_size=batch_sizes[TEST], shuffle=False, num_workers=0)


print(dataset_sizes)
class_names = {0: 'No Cactus', 1: 'Cactus'}

inputs, classes = next(iter(dataloaders[TRAIN]))

n_features = 32 * 32

print(n_features)


{'train': 12250, 'val': 3500, 'test': 1750}
1024


In [114]:
#Load data
X_train = []
y_train = []
for images, labels in dataloaders[TRAIN]:
    images = images.view(-1, n_features)
    print(images.shape)
    X_train.append(images)
    y_train.append(labels)
X_train = torch.cat(X_train, dim=0)
y_train = torch.cat(y_train, dim=0)
X_val = []
y_val = []
for images, labels in dataloaders[VAL]:
    images = images.view(-1, n_features)
    X_val.append(images)
    y_val.append(labels)
X_val = torch.cat(X_val, dim=0)
y_val = torch.cat(y_val, dim=0)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

torch.Size([12250, 1024])
torch.Size([12250, 1024])
torch.Size([12250])
torch.Size([3500, 1024])
torch.Size([3500])


In [115]:
model = LogisticRegression(n_features)
# Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [116]:
train_model(model, criterion, optimizer, 'logistic_regression_grayscale', X_train, y_train, X_val, y_val, num_epochs=1000)

/tmp/ipykernel_69432/2598064429.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_predicted = model(torch.tensor(X_train))


epoch: 100, loss = 0.5691
Accuracy: 0.7460
TP: 2611.0, TN: 0.0, FP: 889.0, FN: 0.0
Precision: 0.7460, Recall: 1.0000
tensor(0.7460)


/tmp/ipykernel_69432/2598064429.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_predicted = model(torch.tensor(X_val))  # no need to call model.forward()


epoch: 200, loss = 0.5616
Accuracy: 0.7460
TP: 2611.0, TN: 0.0, FP: 889.0, FN: 0.0
Precision: 0.7460, Recall: 1.0000
tensor(0.7460)
epoch: 300, loss = 0.5546
Accuracy: 0.7460
TP: 2611.0, TN: 0.0, FP: 889.0, FN: 0.0
Precision: 0.7460, Recall: 1.0000
tensor(0.7460)
epoch: 400, loss = 0.5482
Accuracy: 0.7460
TP: 2611.0, TN: 0.0, FP: 889.0, FN: 0.0
Precision: 0.7460, Recall: 1.0000
tensor(0.7460)
epoch: 500, loss = 0.5422
Accuracy: 0.7460
TP: 2611.0, TN: 0.0, FP: 889.0, FN: 0.0
Precision: 0.7460, Recall: 1.0000
tensor(0.7460)
epoch: 600, loss = 0.5367
Accuracy: 0.7463
TP: 2611.0, TN: 1.0, FP: 888.0, FN: 0.0
Precision: 0.7462, Recall: 1.0000
tensor(0.7463)
epoch: 700, loss = 0.5315
Accuracy: 0.7463
TP: 2611.0, TN: 1.0, FP: 888.0, FN: 0.0
Precision: 0.7462, Recall: 1.0000
tensor(0.7463)
epoch: 800, loss = 0.5267
Accuracy: 0.7463
TP: 2610.0, TN: 2.0, FP: 887.0, FN: 1.0
Precision: 0.7464, Recall: 0.9996
tensor(0.7463)
epoch: 900, loss = 0.5222
Accuracy: 0.7463
TP: 2609.0, TN: 3.0, FP: 886.0, F